<a href="https://colab.research.google.com/github/sunnatillo91/ai-datascience/blob/main/05_ML_04_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Science va Sun'iy Intellekt Praktikum

## 5-MODUL. MACHINE LEARNING

## 4-QADAM. Machine Learning

In [2]:
import pandas as pd
import numpy as np
import sklearn  # scikit-leaarn kutubxonasi

# Onlayn dataset joylashgan manzilni ko'rsatamiz
url = 'http://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true'
df = pd.read_csv(url)

- Ma'lumotlarni train va testga ajratamiz

In [4]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

X_train= train_set.drop("median_house_value", axis=1)
y = train_set["median_house_value"].copy()

X_num=X_train.drop("ocean_proximity", axis=1)

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

#bizga kerak ustunlar indekslari
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
  def __init__(self, add_bedrooms_per_room = True):
    self.add_bedrooms_per_room = add_bedrooms_per_room
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
    population_per_household = X[:, population_ix] / X[:, household_ix]
    if self.add_bedrooms_per_room:
      bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
      return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
    else:
      return np.c_[X, rooms_per_household, population_per_household]

# Pipeline da sonlar

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy="median")),
  ('attribs_adder', CombinedAttributesAdder()),
  ('std_scaler', StandardScaler()),
])

num_pipeline.fit_transform(X_num)

array([[ 1.27258656, -1.3728112 ,  0.34849025, ..., -0.17491646,
         0.05137609, -0.2117846 ],
       [ 0.70916212, -0.87669601,  1.61811813, ..., -0.40283542,
        -0.11736222,  0.34218528],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.08821601,
        -0.03227969, -0.66165785],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ..., -0.60675918,
         0.02030568,  0.99951387],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.40217517,
         0.00707608, -0.79086209],
       [-1.41489815,  0.99543676,  1.85617335, ..., -0.85144571,
        -0.08535429,  1.69520292]])

#FullPipeline, Pipeline da matnlar bilan ishlash

In [7]:
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
  ("num", num_pipeline, num_attribs),
  ("cat", OneHotEncoder(), cat_attribs),
])

In [8]:
X_prepared = full_pipeline.fit_transform(X_train)

In [9]:
X_prepared[0:5,:]

array([[ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
         0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
        -0.2117846 ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.70916212, -0.87669601,  1.61811813,  0.34029326,  0.59309419,
        -0.09890135,  0.6720272 , -0.03584338, -0.40283542, -0.11736222,
         0.34218528,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.44760309, -0.46014647, -1.95271028, -0.34259695, -0.49522582,
        -0.44981806, -0.43046109,  0.14470145,  0.08821601, -0.03227969,
        -0.66165785,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 1.23269811, -1.38217186,  0.58654547, -0.56148971, -0.40930582,
        -0.00743434, -0.38058662, -1.01786438, -0.60001532,  0.07750687,
         0.78303162,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.10855122,  0.5320839 ,  1

# LinearRegression modeli

In [10]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_prepared, y)

LinearRegression()

In [11]:
test_data = X_train.sample(10)
test_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
18394,-121.87,37.27,18.0,3561.0,560.0,1753.0,553.0,5.0292,<1H OCEAN
16927,-122.33,37.58,43.0,1772.0,422.0,1573.0,401.0,2.7474,NEAR OCEAN
10488,-117.68,33.49,16.0,3084.0,724.0,2557.0,690.0,2.8357,<1H OCEAN
8089,-118.21,33.81,43.0,905.0,199.0,764.0,204.0,3.3214,NEAR OCEAN
11115,-117.87,33.84,23.0,1678.0,369.0,912.0,347.0,4.5000,<1H OCEAN


In [12]:
test_labels = y.loc[test_data.index]
test_labels

18394    269400.0
16927    233100.0
10488    106300.0
8089     162200.0
11115    237300.0
17424    111500.0
18688    305600.0
10248    274500.0
18644    251900.0
20505    248000.0
Name: median_house_value, dtype: float64

In [13]:
test_data_prepared = full_pipeline.transform(test_data)
test_labels_predictions = lin_reg.predict(test_data_prepared)
test_labels_predictions

array([257407.3986618 , 200234.55146732, 145556.08785489, 210084.77873045,
       237711.42978358, 208251.91233363, 280872.9531952 , 315305.1406534 ,
       244300.14792871, 295188.20269607])

In [14]:
pd.DataFrame({"Actual": test_labels, "Predicted": test_labels_predictions})

,Actual,Predicted
18394,269400.0,257407.398662
16927,233100.0,200234.551467
10488,106300.0,145556.087855
8089,162200.0,210084.778730
11115,237300.0,237711.429784
17424,111500.0,208251.912334
18688,305600.0,280872.953195
10248,274500.0,315305.140653
18644,251900.0,244300.147929
20505,248000.0,295188.202696


# Modelni baholash

In [15]:
X_test = test_set.drop("median_house_value", axis=1)
X_test.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,INLAND
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,INLAND
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,NEAR BAY
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,<1H OCEAN
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,NEAR OCEAN


In [16]:
y_test = test_set["median_house_value"].copy()
y_test.head()

20046     47700.0
3024      45800.0
15663    500001.0
20484    218600.0
9814     278000.0
Name: median_house_value, dtype: float64

In [17]:
X_test_prepared = full_pipeline.transform(X_test)

In [18]:
y_predicted = lin_reg.predict(X_test_prepared)

## Modelning o'rtacha absolyut xatoligini hisoblash

In [19]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_predicted)
mae

50898.7395349408

## Modelning o'rtacha kvadratik xatoligini hisoblash




In [20]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_predicted)
print("RMSE=", np.sqrt(mse))

RMSE= 72701.32600762138


# Random Forest Modeli

In [21]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(X_prepared, y)

RandomForestRegressor()

In [22]:
y_predicted = forest_reg.predict(X_test_prepared)

In [23]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_predicted)
print("RMSE=", np.sqrt(mse))

RMSE= 50034.95657071726


# Cross-Validation modellarni aniqroq baholash

In [24]:
X = df.drop("median_house_value", axis=1)
y = df["median_house_value"].copy()

X_prepared = full_pipeline.fit_transform(X)

In [27]:
from sklearn.model_selection import cross_val_score

mse_scores = cross_val_score(lin_reg, X_prepared, y, scoring="neg_mean_squared_error", cv=10)


In [28]:
def display_scores(scores):
  print("Scores:", scores)
  print("Mean:", scores.mean())
  print("Standard deviation:", scores.std())

In [29]:
display_scores(np.sqrt(-mse_scores))

Scores: [84183.66301514 61191.52853899 86743.60959739 62286.73445075
 80537.25795828 68918.58661112 52504.86407192 90904.22793667
 77675.08903006 53940.95369716]
Mean: 71888.65149074617
Standard deviation: 13247.67185583078


In [33]:
scores = cross_val_score(forest_reg, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)

In [35]:
display_scores(forest_rmse_scores)

Scores: [96255.12307119 47617.87726608 65168.20349641 56393.97863714
 61001.21290662 60301.62239612 47139.35104516 78660.39995522
 74361.81673077 49448.41034341]
Mean: 63634.79958481233
Standard deviation: 14858.145897147102
